In [12]:
import ee
import math
import geemap # Assuming geemap is installed

# --- Configuration ---
GCP_PROJECT = 'mapbiomas-solos-workspace'
AOI_ASSET_ID = 'projects/mapbiomas-workspace/AUXILIAR/brasil_2km'
SOIL_SAMPLES_ID = 'projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/granulometry/matriz-v010_0_10cm'
# MAPBIOMAS_COLLECTION_ID = 'projects/mapbiomas-workspace/COLECAO9/integracao' # No longer needed for CRS
OUTPUT_SCALE = 30  # meters
MAX_DISTANCE_METERS = 200000 # 200 km (Adjust if needed)

# *** DEFINE TARGET PROJECTED CRS for calculations ***
# Using SIRGAS 2000 / Brazil Polyconic (EPSG:5880) - Projected CRS in Meters
# This is chosen because EPSG:4326 is unsuitable for meter-based scale/distance calculations.
TARGET_CRS = 'EPSG:5880'
print(f"Using Target CRS for calculations: {TARGET_CRS} (Projected, Meters)")

Using Target CRS for calculations: EPSG:5880 (Projected, Meters)


In [13]:
# --- 1. Initialize Earth Engine ---
ee.Initialize(project=GCP_PROJECT)
print(f"EE initialized with project: {GCP_PROJECT}")

EE initialized with project: mapbiomas-solos-workspace


In [14]:
# --- 2. Define Area of Interest ---
print(f"Loading AOI: {AOI_ASSET_ID}")
aoi_asset = ee.FeatureCollection(AOI_ASSET_ID)
aoi_geometry = aoi_asset.union(1) # Merge features into a single geometry

Loading AOI: projects/mapbiomas-workspace/AUXILIAR/brasil_2km


In [15]:
# --- 3. Load and Filter Soil Samples ---
print(f"Loading Soil Samples: {SOIL_SAMPLES_ID}")
soil_points_all = ee.FeatureCollection(SOIL_SAMPLES_ID)
soil_points = soil_points_all.filterBounds(aoi_geometry)
point_count = soil_points.size().getInfo() # Get count after filtering
print(f"Found and filtered {point_count} points within AOI.")

# --- 3b. Visualize Filtered Points ---
if point_count > 0:
    print("\nPreparing visualization of filtered points on AOI...")
    Map_points = geemap.Map()
    Map_points.centerObject(aoi_geometry, 5)
    Map_points.addLayer(ee.Image().paint(aoi_geometry, 0, 2), {'palette': '000000'}, 'AOI Boundary')
    Map_points.addLayer(soil_points, {'color': '0000FF', 'pointSize': 1}, 'Filtered Soil Samples')
    Map_points.addLayerControl()
    print("Displaying filtered points map...")
    display(Map_points)
    print("--- End of points visualization ---")
else:
    print("\nSkipping points visualization as no points were found within the AOI.")


# --- Add constant property (only if points exist) ---
if point_count > 0:
    def add_constant(feat):
      return feat.set({'constant': 1})
    soil_points_with_constant = soil_points.map(add_constant)
else:
    soil_points_with_constant = ee.FeatureCollection([])

Loading Soil Samples: projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/granulometry/matriz-v010_0_10cm
Found and filtered 19864 points within AOI.

Preparing visualization of filtered points on AOI...
Displaying filtered points map...


Map(center=[-10.62347807211283, -53.19353084531423], controls=(WidgetControl(options=['position', 'transparent…

--- End of points visualization ---


In [16]:
# --- 4. Prepare Image for Distance Calculation --- (Step 4 is now this)
if point_count > 0:
    print("\nRasterizing points...")
    # Reproject using the explicitly defined projected CRS
    print(f"Reprojecting to CRS: {TARGET_CRS} with scale {OUTPUT_SCALE}m")
    point_presence_image = soil_points_with_constant.reduceToImage(
        properties=['constant'],
        reducer=ee.Reducer.sum().unweighted()
    ).reproject(
        crs=TARGET_CRS, # Use the defined projected CRS
        scale=OUTPUT_SCALE
    ).unmask(0)
    source_image = point_presence_image.gt(0).Not()


Rasterizing points...
Reprojecting to CRS: EPSG:5880 with scale 30m


In [17]:
# --- 5. Calculate Distance & Get Stats --- (Step 5 is now this)
print("Calculating distance to nearest neighbor...")
neighborhood_pixels = math.ceil(MAX_DISTANCE_METERS / OUTPUT_SCALE)
# Since the image is now projected in meters, units='meters' is correct
distance_image_multiband = source_image.fastDistanceTransform(
    neighborhood=neighborhood_pixels,
    units='meters'
)
distance_image = distance_image_multiband.select('distance')

# --- Print basic info ---
print("\n--- Info about 'distance_image' (before clipping) ---")
try:
    band_names = distance_image.bandNames().getInfo()
    print(f"Band Names: {band_names}")
    # Projection should now match TARGET_CRS
    print(f"Expected CRS: {TARGET_CRS}")
    print(f"Expected Scale (meters): {OUTPUT_SCALE}")
except Exception as e:
    print(f"Could not retrieve basic info: {e}")
print("----------------------------------------------------")

# # --- Calculate and Print Statistics ---
# print("\nCalculating statistics for the distance image (within AOI)...")
# stats_reducer = ee.Reducer.minMax() \
#     .combine(ee.Reducer.mean(), sharedInputs=True) \
#     .combine(ee.Reducer.stdDev(), sharedInputs=True) \
#     .combine(ee.Reducer.percentile([50, 90, 95]), sharedInputs=True) \
#     .combine(ee.Reducer.count(), sharedInputs=True)
# try:
#     stats = distance_image.reduceRegion(
#         reducer=stats_reducer,
#         geometry=aoi_geometry,
#         scale=OUTPUT_SCALE,
#         maxPixels=1e11,
#         tileScale=4
#     )
#     stats_info = stats.getInfo()
#     print("--- Distance Statistics (meters, within AOI) ---")
#     print(f"Min Distance: {stats_info.get('distance_min', 'N/A'):,.2f}")
#     print(f"Max Distance: {stats_info.get('distance_max', 'N/A'):,.2f}")
#     print(f"Mean Distance: {stats_info.get('distance_mean', 'N/A'):,.2f}")
#     print(f"Std Dev Distance: {stats_info.get('distance_stdDev', 'N/A'):,.2f}")
#     print(f"Median (P50) Distance: {stats_info.get('distance_p50', 'N/A'):,.2f}")
#     print(f"90th Percentile Distance: {stats_info.get('distance_p90', 'N/A'):,.2f}")
#     print(f"95th Percentile Distance: {stats_info.get('distance_p95', 'N/A'):,.2f}")
#     print(f"Pixel Count in AOI: {stats_info.get('distance_count', 'N/A'):,}")
#     print("------------------------------------------------")
# except Exception as e:
#     print(f"Could not calculate statistics: {e}")
#     stats_info = None
# --- End Statistics Calculation ---

Calculating distance to nearest neighbor...

--- Info about 'distance_image' (before clipping) ---
Band Names: ['distance']
Expected CRS: EPSG:5880
Expected Scale (meters): 30
----------------------------------------------------


In [18]:
# --- 6. Clip Result --- (Step 6 is now this)
print("\nClipping result to AOI...")
distance_image_clipped = distance_image.clip(aoi_geometry)


Clipping result to AOI...


In [19]:
# --- 7. Visualize DNN Map --- (Step 7 is now this)

# This block assumes the script determined point_count > 0 in earlier steps
# and that 'distance_image_clipped' has been calculated.

print("\nPreparing final DNN map visualization...")
Map_dnn = geemap.Map() # Create a map object
Map_dnn.centerObject(aoi_geometry, 5) # Center on the AOI

# --- Determine Visualization Max ---
# Use a default max, but adjust based on calculated stats if available
vis_max = MAX_DISTANCE_METERS # Default visualization range max
# Check if stats were successfully calculated and contain needed keys
if 'stats_info' in locals() and stats_info and stats_info.get('distance_max') is not None:
  # Optionally cap the visualization max based on stats (e.g., 1.1 * p95)
  # to avoid extreme outliers skewing the color scale too much.
  p95 = stats_info.get('distance_p95')
  # Use the smaller of: calculated max, 1.1*p95 (if p95 exists), 2*default_max
  vis_max = min(stats_info.get('distance_max'), p95 * 1.1 if p95 else MAX_DISTANCE_METERS * 1.1, MAX_DISTANCE_METERS * 2)
  print(f"Using calculated max (capped) for visualization: {vis_max:,.2f} m")
else:
    # Keep the default if stats are not available
    print(f"Using default max for visualization: {vis_max:,.2f} m")
# --- End Determine Visualization Max ---

# Define visualization parameters for the distance layer
dnn_vis_params = {
  'min': 0,
  'max': vis_max, # Use default or adjusted max
  'palette': ['0000FF', '00FFFF', 'FFFF00', 'FF0000'] # Blue=Near -> Red=Far
}
print(f"Visualizing distances from 0 to {dnn_vis_params['max']} meters.")

# Add the clipped distance image layer
Map_dnn.addLayer(
    distance_image_clipped,
    dnn_vis_params,
    f'Distance to Nearest Soil Sample (m)' # Layer name
)
# Add the AOI boundary layer
Map_dnn.addLayer(
    ee.Image().paint(aoi_geometry, 0, 2), # Paint outline
    {'palette': '000000'}, # Black color
    'AOI Boundary' # Layer name
)
# Add layer controls to the map
Map_dnn.addLayerControl()

# Display the map in the output cell
print("Displaying final DNN map...")
display(Map_dnn)

# Note: The 'else' block handling point_count == 0 would appear *outside*
# this specific step in the full script, wrapping steps 4 through 7.


Preparing final DNN map visualization...
Using default max for visualization: 200,000.00 m
Visualizing distances from 0 to 200000 meters.
Displaying final DNN map...


Map(center=[-10.62347807211283, -53.19353084531423], controls=(WidgetControl(options=['position', 'transparent…

In [46]:
# --- 8. Visualize DNN Map ---

# Ensure geemap is imported if running this block standalone
import geemap

# Check if distance_image_clipped likely exists before trying to visualize
# (Assumes it was calculated in prior steps if point_count > 0)
if 'distance_image_clipped' in locals():
    print("\nPreparing final DNN map visualization...")
    # Create a new map object for the final result
    Map_dnn = geemap.Map()
    # Center the map on the Area of Interest
    Map_dnn.centerObject(aoi_geometry, 5)

    # Define visualization parameters for the distance map
    dnn_vis_params = {
      'min': 0,
      # Use the configured max distance for the color scale limit
      'max': MAX_DISTANCE_METERS,
      # Define the color palette (e.g., blue=near, red=far)
      'palette': ['0000FF', '00FFFF', 'FFFF00', 'FF0000']
    }
    print(f"Visualizing distances from 0 to {dnn_vis_params['max']} meters.")

    # Add the calculated distance layer to the map
    Map_dnn.addLayer(
        distance_image_clipped,
        dnn_vis_params,
        f'Distance to Nearest Soil Sample (m)' # Layer name
    )
    # Add the AOI boundary outline for context
    Map_dnn.addLayer(
        ee.Image().paint(aoi_geometry, 0, 2), # Paint outline
        {'palette': '000000'}, # Black color for outline
        'AOI Boundary' # Layer name
    )
    # Add map layer controls (optional)
    Map_dnn.addLayerControl()

    # Display the map in the Jupyter Notebook output
    print("Displaying final DNN map...")
    display(Map_dnn)

else:
    # This part would typically execute if point_count was 0 in the full script
    print("\nSkipping DNN map visualization because the distance image was not generated (likely no input points found).")


Preparing final DNN map visualization...
Visualizing distances from 0 to 200000 meters.
Displaying final DNN map...


Map(center=[-10.62347807211283, -53.19353084531423], controls=(WidgetControl(options=['position', 'transparent…